# Data Preprocessing and Preparation for Analysis

Using the High Frequency Archaeological Survey approach detailed in the [`collection`](../collection/) directory of this replication materials repository, we collect many small CSV files. CSV files are fast and easy to write from our Lambda workers, but are quite slow to read (and perform analytical operations on using Spark). Therefore, it is better if we convert these CSVs into a smaller number of Parquet partitions -- a column-based file format that can be processed faster in Spark. We can additionally define our data schema here, which will make it easier to analyze the aggregate data for the month in the [`01_analysis.ipynb`](./01_analysis.ipynb) code in this same directory. The function below accomplishes all of these goals -- converting our CSVs into Parquet (divided into 24 partitions by default).

Note that this code is intended to be run on an AWS EMR cluster using a PySpark kernel, which can be done via a "Workspace" in AWS [EMR Studio](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-studio.html). The following has been tested on an EMR cluster (1 primary node, 6 core nodes -- all m5.xlarge EC2 instances) using EMR release label 6.2.0 with Hadoop (3.2.1), Hive (3.1.2), JupyterEnterpriseGateway (2.1.0), Livy (0.7.0), and Spark (3.0.1) installed. For detailed instructions on how to launch and configure an equivalent cluster, consult [the AWS EMR documentation](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-configure-apps-create-cluster.html?icmpid=docs_emr_help_panel
).

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

def convert_csv_to_parquet(input_path, output_path, n_partitions=24):
    # Define schema for data
    schema = T.StructType([
        T.StructField("city", T.StringType()),
        T.StructField("category", T.StringType()),
        T.StructField("post_id", T.StringType()),
        T.StructField("time_posted", T.TimestampType()),
        T.StructField("last_updated", T.TimestampType()),
        T.StructField("time_downloaded", T.TimestampType()),
        T.StructField("title", T.StringType()),
        T.StructField("body_text", T.StringType()),
        T.StructField("latitude", T.DoubleType()),
        T.StructField("longitude", T.DoubleType()),
        T.StructField("image_s3_keys", T.StringType())
    ])

    # Read all CSVs in Input Path file range
    # note that all timestamps are converted to UTC regardless of original time zone
    data = spark.read.csv(input_path, 
                          header=True, 
                          schema=schema,
                          timestampFormat="yyyy-MM-dd'T'HH:mm:ssZ",
                          quote="\"",
                          escape="\"") \
                .withColumn("image_s3_keys", F.split("image_s3_keys", "\,"))

    # Repartition data into a smaller number of partitions and write to parquet for
    # faster analytical processing
    data.repartition(n_partitions).write.parquet(output_path)

We can then provide an input and output path and let our function process our data for us. Note that for large input path file ranges (and depending on how much memory is available in the EMR cluster), it might make sense to split the preprocessing up into smaller batches and use the function to incrementally process these batches (e.g. in approximately ten-day increments -- such as first processing the input path `/2023_02_0*/*/*/*.csv` for February 1-9, then `/2023_02_1*/*/*/*.csv` for February 10-19, and finally `/2023_02_2*/*/*/*.csv` for February 20-28). The resulting parquet partitions can be further repartitioned if the partitions are too small/large.

In [ ]:
input_path = 's3://craigslist-post-data/data/zip/2023_02_*/*/*/*.csv'
output_path = 's3://craigslist-post-data/02_2023'
convert_csv_to_parquet(input_path, output_path)